In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import nltk
import re
import string 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import array
from numpy import asarray
from numpy import zeros
import tensorflow as tf
from tensorflow import keras
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing import text,sequence
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout

In [ ]:
pip install -q -U keras-tuner

In [ ]:
import keras_tuner as kt

In [ ]:
train_data = pd.read_csv('../input/fake-news/train.tsv',sep="\t")

In [ ]:
train_data.columns=['id','label','statement','subjects','speaker','speaker_job_title','state_info','party_affiliation','count_1','count_2','count_3','count_4','count_5','context']

In [ ]:
for i in range(0,len(train_data)):
  if((train_data['label'][i]=='true')):
    train_data['label'][i]=1
  else:
    train_data['label'][i]=0

In [ ]:
data=train_data
data.head()

In [ ]:
data['text']= data['statement'] + " " + data['subjects'] + " " + data['context']
del data['statement']
del data['id']
del data['subjects']
del data['context']
del data['speaker']
del data['speaker_job_title']
del data['state_info']
del data['party_affiliation']
del data['count_1']
del data['count_2']
del data['count_3']
del data['count_4']
del data['count_5']
data.head()

In [ ]:
data.dropna()
data.fillna("", inplace=True)

In [ ]:
pip install bs4

In [ ]:
import nltk
nltk.download('punkt')
from bs4 import BeautifulSoup
nltk.download("stopwords")   
from nltk.corpus import stopwords
nltk.download('wordnet')

In [ ]:
def remove_characters(text):
    return re.sub("[^a-zA-Z]"," ",text)

#Removal of stopwords 
def remove_stopwords_and_lemmatization(text):
    final_text = []
    text = text.lower()
    text = nltk.word_tokenize(text)
    
    for word in text:
        if word not in set(stopwords.words('english')):
            lemma = nltk.WordNetLemmatizer()
            word = lemma.lemmatize(word) 
            final_text.append(word)
    return " ".join(final_text)

#Total function
def cleaning(text):
    text = remove_characters(text)
    text = remove_stopwords_and_lemmatization(text)
    return text

#Apply function on text column
data['text']=data['text'].apply(cleaning)

In [ ]:
data=data.sample(frac=1)

**Training when data is 5% labled**

In [ ]:
X_train1, X_test, y_train1, y_test = train_test_split(data['text'], data['label'],test_size=0.1 ,random_state=42)

In [ ]:
X_lvl, X_unl, y_lvl, y_unl = train_test_split(X_train1,y_train1,test_size=0.80,random_state=0)

In [ ]:
max_features = 3000
maxlen = 50

In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_lvl)
tokenized_train = tokenizer.texts_to_sequences(X_lvl)
X_lvl = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

In [ ]:
tokenized_test = tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [ ]:
tokenized_unl = tokenizer.texts_to_sequences(X_unl)
X_unl = sequence.pad_sequences(tokenized_unl, maxlen=maxlen)

In [ ]:
batch_size = 128

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers

def build_model(hp):
    model = Sequential()
    neurons = hp.Choice('units', values=[32,64, 128,256])
    model.add(Embedding(max_features, output_dim=100, input_length=maxlen, trainable=False))
    model.add(LSTM(units=neurons , return_sequences = True , recurrent_dropout =0.25,dropout=0.25))
    model.add(LSTM(units=neurons , recurrent_dropout = 0.25 , dropout = 0.25))
    model.add(Dense(units=neurons , activation = 'relu'))
    model.add(Dense(1, activation='sigmoid'))
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3])
    model.compile(tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=5,
                     factor=3)

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner.search(X_lvl,y_lvl, epochs=20, validation_split=0.2, callbacks=[stop_early])
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
maxitr=3
itera=0
tl=len(data)-len(X_test)
ixll=len(X_lvl)
ixul=len(X_unl)
X_train=X_lvl
y_train=y_lvl
model=tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, validation_split=0.3, epochs=100, batch_size=batch_size, shuffle=True, verbose = 1)
while(len(X_unl)>0):
    pred1=(model.predict(X_unl))
    pred0=1-pred1
    df_pred_prob = pd.DataFrame([])
    df_pred_prob['prob_0'] = (pred0).tolist()
    df_pred_prob['prob_1'] = (pred1).tolist()
    df_pred_prob['text']=X_unl.tolist()
    df=df_pred_prob
    yln=[]
    Xln=[]
    Xun=[]
    for i in range(len(X_unl)):
        if((df['prob_0'][i][0])>0.75):
            yln=yln+[0]
            Xln=Xln+[df['text'][i]]
        elif((df['prob_1'][i][0])>0.75):
            yln=yln+[1]
            Xln=Xln+[df['text'][i]]
        else:
            Xun=Xun+[df['text'][i]]
    
    y_train=np.concatenate((y_train,yln),axis=0)
    X_train=np.concatenate((X_train,Xln),axis=0)
    ixll=len(X_train)
    print("length of train data:=")
    print(ixll)
    model=tuner.hypermodel.build(best_hps)
    history = model.fit(X_train, y_train, validation_split=0.3, epochs=50, batch_size=batch_size, shuffle=True, verbose = 1)
    X_unl = np.array(Xun)
    print("length of remaining unlabled data:=")
    print(len(X_unl))
    itera=itera+1
    if (itera==maxitr):
        break
    

In [ ]:
test_res=(model.predict(X_test) > 0.5).astype("int32")
test_res=test_res.flatten()
y_test_c = y_test.to_numpy()

In [ ]:
count=0
for i in range(0,len(y_test_c)):
    if(y_test_c[i]==test_res[i]):
        count=count+1
print(count)

In [ ]:
accuracy=(count/len(test_res))*100

In [ ]:
print("Accuracy at 20% labled data",accuracy)

In [ ]:
print(classification_report(y_test,test_res, target_names = ['Fake','Real']))